### Greedy algorithm for reducing redundant moves

In [2]:
import json
import copy

import numpy as np
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# load sample solutions
sample_submission_arr = pd.read_csv("./puzzles/sample_submission.csv").to_numpy()
# load all the puzzle's start and end positions
puzzles_arr = pd.read_csv("./puzzles/puzzles.csv").to_numpy()

puzzle_solutions = {}

for puzzle, solution in zip(puzzles_arr, sample_submission_arr):
    puzzle_name = puzzle[1]
    idx = puzzle[0]
    steps = solution[1].split(".")
    puzzle_solutions[(puzzle_name, idx)] = steps

print(list(puzzle_solutions.items())[0])

(('cube_2/2/2', 0), ['r1', '-f1'])


In [4]:
# some low hanging fruits are:
# cancel pairs of inverses, e.g. (f0, -f0)
# cancel 4 rotations
# change 3 rotations to 1 rotation e.g. (f0, f0, f0 == -f0)


def get_inverse_move(move_name):
    if move_name.startswith("-"):
        return move_name[1:]
    else:
        return f"-{move_name}"


def reduce_subsequence(subsequence):
    """Given a subsequence of parallel moves, return the reduced moves"""
    subsequence.sort()

    out = []
    curr_count, curr_move = 0, ""
    # reduce 3 or 4 in a row:
    for move in subsequence + ["x"]:
        # a dummy item 'x' to make loop run once more for leftover items
        if move != curr_move:
            if curr_count == 3:
                out.append(get_inverse_move(curr_move))
            else:
                out.extend([curr_move] * curr_count)
            curr_count, curr_move = 1, move
        elif move == curr_move:
            curr_count += 1
        if curr_count == 4:
            # skip entirely, don't append to out
            curr_count = 0

    counts = Counter(out)

    out = []
    for move, count in counts.items():
        inv_move = get_inverse_move(move)
        inv_count = counts.get(inv_move, 0)
        if count > inv_count:
            out.extend([move] * (count - inv_count))
        else:
            out.extend([inv_move] * (inv_count - count))
        if inv_move in counts.keys():
            counts[inv_move] = 0
        counts[move] = 0
    return out

def iterate_reduce_subsequence(subsequence):
    prev = []
    curr = subsequence
    while prev != curr:
        prev = curr[:]
        curr = reduce_subsequence(curr)

    return curr

subsequence = ["f0", "f0", "f0", "f0", "f1", "f1", "f1", "-f1", "-f1", "-f0"]
# subsequence = ["f0"]
print(iterate_reduce_subsequence(subsequence))

['-f0', 'f1']


In [12]:
def remove_prefix(text, prefix):
    return text[text.startswith(prefix) and len(prefix) :]


def reduce_sequence(sequence):
    out = []
    curr_face = ""
    curr_subsequence = []
    for move in sequence + ["x"]:
        face = remove_prefix(move, "-")[0]
        if face != curr_face:
            out.extend(iterate_reduce_subsequence(curr_subsequence))
            curr_subsequence = []
        curr_face = face
        curr_subsequence.append(move)

    return out


def iterate_reduce_sequence(sequence):
    # return reduce_sequence(sequence)
    prev = []
    curr = sequence
    while prev != curr:
        prev = curr[:]
        curr = reduce_sequence(curr)
    return curr


# iterate_reduce_sequence(["f0", "f0", "f0", "f0", "f1", "f1", "f1", "-f1", "-f1", "-f0"])

old_total = 0
new_total = 0

out = []
for puzzle, sequence in tqdm((puzzle_solutions.items())):
    puzzle_name = puzzle[0].split("_")[0]
    idx = puzzle[1]

    old_total += len(sequence)
    if puzzle_name == "cube":
        reduced_sequence = iterate_reduce_sequence(sequence)
    else:
        reduced_sequence = sequence
    new_total += len(reduced_sequence)

    out.append((".".join(reduced_sequence)))

print(old_total)
print(new_total)

100%|██████████| 398/398 [00:03<00:00, 108.48it/s]

1220590
1206332


In [14]:
out_df = pd.DataFrame(out)
out_df.to_csv('./solutions/greedy_cubes.csv')